il faut avoir le "présentation_champions_complets.csv" sinon l'echec sera cuisant 

In [ ]:
import requests
import polars as pl
import pandas as pd # On a besoin de Pandas pour le ML et la compatibilité
from sklearn.neighbors import NearestNeighbors
from sklearn.preprocessing import StandardScaler
import joblib

def get_lol_champions_data():
    print("Récupération de la version du patch...")
    try:
        version_url = "https://ddragon.leagueoflegends.com/api/versions.json"
        latest_version = requests.get(version_url).json()[0]
        print(f"Patch détecté : {latest_version}")
    except Exception as e:
        print(f"Erreur version : {e}")
        return None

    url = f"https://ddragon.leagueoflegends.com/cdn/{latest_version}/data/en_US/champion.json"
    print(f"Téléchargement des champions depuis : {url}")
    
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Erreur téléchargement (Status: {response.status_code})")
        return None

    data = response.json()
    champions_dict = data['data']

    rows = []
    for champ_name, details in champions_dict.items():
        stats = details['stats']
        info = details['info']
        
        # Construction de l'URL de l'image de chargement (pour l'affichage Streamlit)
        image_url = f"https://ddragon.leagueoflegends.com/cdn/img/champion/loading/{champ_name}_0.jpg"

        rows.append({
            "Name": champ_name,
            "Class": details['tags'][0],
            "Image_Loading": image_url,  #pour l'affichage

            "AttackRange": stats['attackrange'],
            "MoveSpeed": stats['movespeed'],
            "HP": stats['hp'],
            "HPperLevel": stats['hpperlevel'],
            
            # Attention aux minuscules 
            "hpregen": stats['hpregen'],
            "hpregenperlevel": stats['hpregenperlevel'],
            
            "Armor": stats['armor'],
            "ArmorperLevel": stats['armorperlevel'],
            
            "AttackDamage": stats['attackdamage'],
            "AttackDamageperLevel": stats['attackdamageperlevel'],
            
            "attackspeed": stats['attackspeed'],
            "attackspeedperlevel": stats['attackspeedperlevel'],
            
            "Spellblock": stats['spellblock'],
            "spellblockperlevel": stats['spellblockperlevel'],

            "Prv_attack": info['attack'],
            "Prv_defense": info['defense'],
            "Prv_magic": info['magic']
        })

    # Conversion directe en Pandas pour Scikit-Learn
    return pd.DataFrame(rows)

# programme principal

if __name__ == "__main__":
    
    # 1. Extraction
    df = get_lol_champions_data()
    
    if df is not None:
        print(f"📊 Données chargées : {len(df)} champions.")
        
        # 2 les colonnes ciblées pour le ML
        FEATURES = [
            "HP", "HPperLevel", 
            "MoveSpeed", 
            "Armor", "ArmorperLevel", 
            "Spellblock", "spellblockperlevel", 
            "AttackRange", 
            "hpregen", "hpregenperlevel", 
            "AttackDamage", "AttackDamageperLevel", 
            "attackspeed", "attackspeedperlevel", 
            "Prv_attack", "Prv_defense", "Prv_magic"
        ]

        # 3. Préparation
        X = df[FEATURES].fillna(0)
        
        # 4. Standardisation
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)

        # 5. Entraînement KNN
        knn = NearestNeighbors(n_neighbors=4, metric='euclidean')
        knn.fit(X_scaled)

        # 6. Sauvegarde
        to_save = {
            'model': knn,
            'scaler': scaler,
            'data': df,       # Contient maintenant la colonne "Image_Loading"
            'features': FEATURES
        }

        joblib.dump(to_save, 'knn_data.pkl')
        print(" Fichier 'knn_data.pkl' généré avec succès !")
        print(" App Streamlit fonctionelle ( j'espère)")

Récupération de la version du patch...
Patch détecté : 16.2.1
Téléchargement des champions depuis : https://ddragon.leagueoflegends.com/cdn/16.2.1/data/en_US/champion.json
📊 Données chargées : 172 champions.
 Fichier 'knn_data.pkl' généré avec succès !
 Tu peux maintenant relancer ton app Streamlit.
